# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

Purpose: Import the necessary libraries for processing data and interacting with Cassandra.

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

Purpose: Locate and prepare the dataset for processing.

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

Purpose: Merge all event data into a single denormalized CSV file for Cassandra queries.

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


Creating a Cluster and Session
Cluster: Establishes a connection to the Cassandra instance (default address: 127.0.0.1).

Session: Opens a channel to execute queries on the Cassandra database.

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [8]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS lf_keyspace
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")
for row in rows:
    print(row.keyspace_name)

system_auth
system_schema
system_distributed
system
system_traces
lf_keyspace


Set Keyspace: Activates lf_keyspace for all queries.

In [9]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('lf_keyspace')


# Part II: Apache Cassandra Data Modeling

# Query 1: Fetch Artist, Song Title, and Length
Objective: Retrieve the artist, song title, and song's length during sessionId = 338 and itemInSession = 4.

In [10]:
# Creates table user_session_history
session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_history (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY ((sessionId), itemInSession)
    );
""")
print("Table for Query 1 created!")

Table for Query 1 created!


# Data Insertion:
Insert the data into the user_session_history table from the CSV file.

In [11]:
# Insert data to table user_session_history 
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skip header
    for line in csvreader:
        query = """
        INSERT INTO user_session_history (sessionId, itemInSession, artist, song, length)
        VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query, (
            int(line[8]),   # sessionId
            int(line[3]),   # itemInSession
            line[0],        # artist
            line[9],        # song
            float(line[5])  # length
        ))

# Query 1 execution

In [12]:
# Query to fetch the artist, song title, and song length for sessionId = 338 and itemInSession = 4, 
# retrieving specific content accessed during a session.

rows = session.execute("""
    SELECT artist, song, length
    FROM user_session_history
    WHERE sessionId = 338 AND itemInSession = 4;
""")
for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, Length: {row.length}")


Artist: Faithless, Song: Music Matters (Mark Knight Dub), Length: 495.30731201171875


# Query 2: Fetch Artist, Song (Sorted by ItemInSession), and User Details
Objective: Retrieve the artist, song title (sorted by itemInSession), and the user's first and last name for a specific user (userId = 10) during a specific session (sessionId = 182).

In [13]:
# Creates Table session_songs
session.execute("""
    CREATE TABLE IF NOT EXISTS session_songs (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    );
""")
print("Table for Query 2 created!")


Table for Query 2 created!


# Data Insertion
Insert the data into the session_songs table from the CSV file.

In [ ]:
# Insert data to Table session_songs
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skip header
    for line in csvreader:
        query = """
        INSERT INTO session_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query, (
            int(line[10]),  # userId
            int(line[8]),   # sessionId
            int(line[3]),   # itemInSession
            line[0],        # artist
            line[9],        # song
            line[1],        # firstName
            line[4]         # lastName
        ))

# Query 2 execution

In [ ]:
# Query to fetch the artist, song title (sorted by itemInSession), and user's first and last name 
# for userId = 10 and sessionId = 182, providing a timeline of the user's session activity.

rows = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM session_songs
    WHERE userId = 10 AND sessionId = 182
    ORDER BY itemInSession;
""")
for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, User: {row.firstname} {row.lastname}")


# Query 3: Fetch Users Who Listened to a Specific Song
Objective: Retrieve the first and last names of all users who listened to the song 'All Hands Against His Own'.

In [ ]:
# Create the table for Query 3
session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners (
        song text,
        userId int,
        firstName text,
        lastName text,
        PRIMARY KEY (song, userId)
    );
""")
print("Table for Query 3 created!")



# Data Insertion
Insert the data into the song_listeners table from the CSV file.

In [ ]:
#Insert data to the Table song_listeners
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skip header
    for line in csvreader:
        query = """
        INSERT INTO song_listeners (song, userId, firstName, lastName)
        VALUES (%s, %s, %s, %s)
        """
        session.execute(query, (
            line[9],        # song
            int(line[10]),  # userId
            line[1],        # firstName
            line[4]         # lastName
        ))


# Query 3 execution

In [ ]:
# Query 3: Get users who listened to the song 'All Hands Against His Own'
rows = session.execute("""
    SELECT firstName, lastName
    FROM song_listeners
    WHERE song = 'All Hands Against His Own';
""")
for row in rows:
    print(f"User: {row.firstname} {row.lastname}")


# Drop the tables before closing out the sessions

In [ ]:
# Drop the tables
session.execute("DROP TABLE IF EXISTS session_songs;")
session.execute("DROP TABLE IF EXISTS user_session_history;")
session.execute("DROP TABLE IF EXISTS song_listeners;")

# Confirm the tables have been dropped
print("Tables dropped successfully!")

# Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()